In [1]:
from keras.layers import Dense
from keras.layers import Conv1D
from keras.layers import Flatten
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import MaxPool1D
from keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd
from keras.layers import Dropout
from keras.layers import SpatialDropout1D
from keras.optimizers import Adam,SGD
from keras.layers import BatchNormalization
from keras.layers import GaussianDropout

Using TensorFlow backend.


In [7]:

visibile = Input(shape=(11,94))
dropout_1 = SpatialDropout1D(0.3)(visibile)

conv_11 = Conv1D(filters=30,kernel_size=3,padding='same',data_format='channels_first',activation='relu')(dropout_1)
#bn_11 = BatchNormalization()(conv_11)
conv_12 = Conv1D(filters=30,kernel_size=3,padding='same',data_format='channels_first',activation='relu')(conv_11)
#bn_12 = BatchNormalization()(conv_12)
spatial_dropout_1 = SpatialDropout1D(0.3)(conv_12)
max_pool_1 = MaxPool1D(pool_size=2,strides=2,data_format='channels_first')(spatial_dropout_1)

conv_21 = Conv1D(filters=30,kernel_size=6,padding='same',data_format='channels_first',activation='relu')(max_pool_1)
#conv_21 = Conv1D(filters=30,kernel_size=6,padding='same',data_format='channels_first',activation='relu')(max_pool_1)
#bn_21 = BatchNormalization()(conv_21)
conv_22 = Conv1D(filters=30,kernel_size=6,padding='same',data_format='channels_first',activation='relu')(conv_21)
#bn_22 = BatchNormalization()(conv_22)
spatial_dropout_2 = SpatialDropout1D(0.3)(conv_22)
max_pool_2 = MaxPool1D(pool_size=2,strides=2,data_format='channels_first')(spatial_dropout_2)

conv_31 = Conv1D(filters=30,kernel_size=9,padding='same',data_format='channels_first',activation='relu')(max_pool_2)
#conv_31 = Conv1D(filters=30,kernel_size=9,padding='same',data_format='channels_first',activation='relu')(max_pool_2)
conv_32 = Conv1D(filters=30,kernel_size=9,padding='same',data_format='channels_first',activation='relu')(conv_31)
spatial_dropout_3 = SpatialDropout1D(0.3)(conv_32)
max_pool_3 = MaxPool1D(pool_size=2,strides=2,data_format='channels_first')(spatial_dropout_3)

conv_41 = Conv1D(filters=30,kernel_size=12,padding='same',data_format='channels_first',activation='relu')(max_pool_3)
#conv_41 = Conv1D(filters=30,kernel_size=12,padding='same',data_format='channels_first',activation='relu')(conv_32)
conv_42 = Conv1D(filters=30,kernel_size=12,padding='same',data_format='channels_first',activation='relu')(conv_41)
spatial_dropout_4 = SpatialDropout1D(0.3)(conv_42)
max_pool_4 = MaxPool1D(pool_size=2,strides=2,data_format='channels_first')(spatial_dropout_4)

#conv_51 = Conv1D(filters=40,kernel_size=3,padding='same',data_format='channels_first',activation='relu')(max_pool_4)
#conv_52 = Conv1D(filters=40,kernel_size=3,padding='same',data_format='channels_first',activation='relu')(conv_51)
#spatial_dropout_2 = SpatialDropout1D(0.5)(conv_22)
#max_pool_5 = MaxPool1D(pool_size=2,strides=2,data_format='channels_first')(conv_52)

flat_1 = Flatten()(max_pool_4)
dropout_2 = Dropout(0.2)(flat_1)
#dropout_2 = GaussianDropout(0.2)(flat_1)

dense_1 = Dense(40,activation='relu')(dropout_2)
#bn_3 = BatchNormalization()(dense_1)
dropout_3 = Dropout(0.2)(dense_1)
#dropout_3 = GaussianDropout(0.2)(dense_1)

dense_2 = Dense(20,activation='relu')(dropout_3)
#bn_4 = BatchNormalization()(dense_2)
dropout_4 = Dropout(0.2)(dense_2)
#dropout_4 = GaussianDropout(0.2)(dense_2)

output = Dense(1,activation='sigmoid')(dropout_4)

model = Model(inputs=visibile, outputs=output)

print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 11, 94)            0         
_________________________________________________________________
spatial_dropout1d_6 (Spatial (None, 11, 94)            0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 30, 94)            1020      
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 30, 94)            2730      
_________________________________________________________________
spatial_dropout1d_7 (Spatial (None, 30, 94)            0         
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 30, 47)            0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 30, 47)            5430      
__________

In [6]:
early_stopping_monitor = EarlyStopping(patience=500)



In [4]:
X_train = np.load('X_train.npy')
y_train = np.load('train_labels.npy')
p = np.random.permutation(len(X_train))
X_train = X_train[p,:,:]
y_train = y_train[p]
print(X_train.shape)
print(y_train.shape)

(498, 11, 94)
(498,)


In [8]:
optimizer = Adam(lr=0.00005)
#optimizer = SGD(lr= 0.00001, momentum=0.9)
model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train,y_train,validation_split=0.1, epochs=2000, callbacks=[early_stopping_monitor], batch_size=16)
#model.fit(X_train,y_train,validation_split=0.1, epochs=100)

Train on 448 samples, validate on 50 samples
Epoch 1/2000
448/448 [==============================] - 1s 2ms/step - loss: 3.3596 - acc: 0.5402 - val_loss: 0.7675 - val_acc: 0.4200
Epoch 2/2000
448/448 [==============================] - 0s 382us/step - loss: 2.2505 - acc: 0.5469 - val_loss: 0.6782 - val_acc: 0.6000
Epoch 3/2000
448/448 [==============================] - 0s 378us/step - loss: 2.1137 - acc: 0.4866 - val_loss: 0.6806 - val_acc: 0.6600
Epoch 4/2000
448/448 [==============================] - 0s 376us/step - loss: 1.7658 - acc: 0.4754 - val_loss: 0.6826 - val_acc: 0.6600
Epoch 5/2000
448/448 [==============================] - 0s 376us/step - loss: 1.3899 - acc: 0.5312 - val_loss: 0.6800 - val_acc: 0.6000
Epoch 6/2000
448/448 [==============================] - 0s 378us/step - loss: 1.2961 - acc: 0.5112 - val_loss: 0.6737 - val_acc: 0.6400
Epoch 7/2000
448/448 [==============================] - 0s 376us/step - loss: 1.1898 - acc: 0.5000 - val_loss: 0.6760 - val_acc: 0.6400
Epoch

448/448 [==============================] - 0s 377us/step - loss: 0.7048 - acc: 0.5558 - val_loss: 0.6877 - val_acc: 0.6400
Epoch 61/2000
448/448 [==============================] - 0s 377us/step - loss: 0.7037 - acc: 0.5312 - val_loss: 0.6872 - val_acc: 0.6600
Epoch 62/2000
448/448 [==============================] - 0s 377us/step - loss: 0.6960 - acc: 0.5312 - val_loss: 0.6877 - val_acc: 0.6600
Epoch 63/2000
448/448 [==============================] - 0s 376us/step - loss: 0.6981 - acc: 0.5580 - val_loss: 0.6864 - val_acc: 0.7200
Epoch 64/2000
448/448 [==============================] - 0s 376us/step - loss: 0.6986 - acc: 0.5357 - val_loss: 0.6867 - val_acc: 0.7600
Epoch 65/2000
448/448 [==============================] - 0s 378us/step - loss: 0.6951 - acc: 0.5536 - val_loss: 0.6865 - val_acc: 0.7400
Epoch 66/2000
448/448 [==============================] - 0s 376us/step - loss: 0.7000 - acc: 0.5246 - val_loss: 0.6860 - val_acc: 0.7600
Epoch 67/2000
448/448 [==============================] 

Epoch 120/2000
448/448 [==============================] - 0s 399us/step - loss: 0.6914 - acc: 0.5893 - val_loss: 0.6789 - val_acc: 0.7800
Epoch 121/2000
448/448 [==============================] - 0s 398us/step - loss: 0.6496 - acc: 0.6317 - val_loss: 0.6786 - val_acc: 0.7800
Epoch 122/2000
448/448 [==============================] - 0s 386us/step - loss: 0.6769 - acc: 0.5848 - val_loss: 0.6790 - val_acc: 0.7800
Epoch 123/2000
448/448 [==============================] - 0s 375us/step - loss: 0.6520 - acc: 0.6228 - val_loss: 0.6778 - val_acc: 0.7600
Epoch 124/2000
448/448 [==============================] - 0s 376us/step - loss: 0.6723 - acc: 0.5915 - val_loss: 0.6780 - val_acc: 0.7800
Epoch 125/2000
448/448 [==============================] - 0s 375us/step - loss: 0.6698 - acc: 0.5915 - val_loss: 0.6779 - val_acc: 0.7400
Epoch 126/2000
448/448 [==============================] - 0s 375us/step - loss: 0.6532 - acc: 0.6138 - val_loss: 0.6768 - val_acc: 0.7800
Epoch 127/2000
448/448 [==========

448/448 [==============================] - 0s 375us/step - loss: 0.6428 - acc: 0.6049 - val_loss: 0.6672 - val_acc: 0.7800
Epoch 180/2000
448/448 [==============================] - 0s 374us/step - loss: 0.6360 - acc: 0.6161 - val_loss: 0.6676 - val_acc: 0.7600
Epoch 181/2000
448/448 [==============================] - 0s 376us/step - loss: 0.6423 - acc: 0.6250 - val_loss: 0.6660 - val_acc: 0.7600
Epoch 182/2000
448/448 [==============================] - 0s 376us/step - loss: 0.6540 - acc: 0.6138 - val_loss: 0.6676 - val_acc: 0.7600
Epoch 183/2000
448/448 [==============================] - 0s 376us/step - loss: 0.6321 - acc: 0.6384 - val_loss: 0.6649 - val_acc: 0.7600
Epoch 184/2000
448/448 [==============================] - 0s 376us/step - loss: 0.6322 - acc: 0.6362 - val_loss: 0.6647 - val_acc: 0.7600
Epoch 185/2000
448/448 [==============================] - 0s 375us/step - loss: 0.6403 - acc: 0.6272 - val_loss: 0.6648 - val_acc: 0.7800
Epoch 186/2000
448/448 [=========================

448/448 [==============================] - 0s 397us/step - loss: 0.6445 - acc: 0.6384 - val_loss: 0.6656 - val_acc: 0.7600
Epoch 239/2000
448/448 [==============================] - 0s 399us/step - loss: 0.6334 - acc: 0.6473 - val_loss: 0.6654 - val_acc: 0.7600
Epoch 240/2000
448/448 [==============================] - 0s 383us/step - loss: 0.6485 - acc: 0.6362 - val_loss: 0.6668 - val_acc: 0.7600
Epoch 241/2000
448/448 [==============================] - 0s 374us/step - loss: 0.6372 - acc: 0.6451 - val_loss: 0.6661 - val_acc: 0.7600
Epoch 242/2000
448/448 [==============================] - 0s 375us/step - loss: 0.6336 - acc: 0.6272 - val_loss: 0.6684 - val_acc: 0.7600
Epoch 243/2000
448/448 [==============================] - 0s 385us/step - loss: 0.6362 - acc: 0.6339 - val_loss: 0.6676 - val_acc: 0.7600
Epoch 244/2000
448/448 [==============================] - 0s 377us/step - loss: 0.6275 - acc: 0.6362 - val_loss: 0.6677 - val_acc: 0.7600
Epoch 245/2000
448/448 [=========================

...

448/448 [==============================] - 0s 374us/step - loss: 0.4106 - acc: 0.7924 - val_loss: 0.3675 - val_acc: 0.8800
Epoch 1714/2000
448/448 [==============================] - 0s 375us/step - loss: 0.4270 - acc: 0.8214 - val_loss: 0.3696 - val_acc: 0.8800
Epoch 1715/2000
448/448 [==============================] - 0s 375us/step - loss: 0.4116 - acc: 0.7946 - val_loss: 0.3691 - val_acc: 0.8800
Epoch 1716/2000
448/448 [==============================] - 0s 375us/step - loss: 0.4025 - acc: 0.8013 - val_loss: 0.3699 - val_acc: 0.8800
Epoch 1717/2000
448/448 [==============================] - 0s 375us/step - loss: 0.4176 - acc: 0.7946 - val_loss: 0.3790 - val_acc: 0.8800
Epoch 1718/2000
448/448 [==============================] - 0s 374us/step - loss: 0.4188 - acc: 0.7902 - val_loss: 0.3735 - val_acc: 0.8800
Epoch 1719/2000
448/448 [==============================] - 0s 376us/step - loss: 0.4101 - acc: 0.8080 - val_loss: 0.3681 - val_acc: 0.8800
Epoch 1720/2000
448/448 [==================

448/448 [==============================] - 0s 403us/step - loss: 0.4135 - acc: 0.7969 - val_loss: 0.3446 - val_acc: 0.9000
Epoch 1773/2000
448/448 [==============================] - 0s 401us/step - loss: 0.4024 - acc: 0.8080 - val_loss: 0.3377 - val_acc: 0.9000
Epoch 1774/2000
448/448 [==============================] - 0s 399us/step - loss: 0.3975 - acc: 0.7924 - val_loss: 0.3397 - val_acc: 0.8800
Epoch 1775/2000
448/448 [==============================] - 0s 396us/step - loss: 0.4040 - acc: 0.7991 - val_loss: 0.3295 - val_acc: 0.9000
Epoch 1776/2000
448/448 [==============================] - 0s 374us/step - loss: 0.3966 - acc: 0.8192 - val_loss: 0.3374 - val_acc: 0.9000
Epoch 1777/2000
448/448 [==============================] - 0s 377us/step - loss: 0.3817 - acc: 0.8147 - val_loss: 0.3339 - val_acc: 0.9000
Epoch 1778/2000
448/448 [==============================] - 0s 376us/step - loss: 0.3963 - acc: 0.7924 - val_loss: 0.3309 - val_acc: 0.9000
Epoch 1779/2000
448/448 [==================

448/448 [==============================] - 0s 385us/step - loss: 0.3765 - acc: 0.8147 - val_loss: 0.3413 - val_acc: 0.8800
Epoch 1832/2000
448/448 [==============================] - 0s 383us/step - loss: 0.3779 - acc: 0.7991 - val_loss: 0.3361 - val_acc: 0.9000
Epoch 1833/2000
448/448 [==============================] - 0s 384us/step - loss: 0.3719 - acc: 0.8147 - val_loss: 0.3305 - val_acc: 0.8800
Epoch 1834/2000
448/448 [==============================] - 0s 405us/step - loss: 0.4244 - acc: 0.8125 - val_loss: 0.3412 - val_acc: 0.8600
Epoch 1835/2000
448/448 [==============================] - 0s 411us/step - loss: 0.4250 - acc: 0.7991 - val_loss: 0.3491 - val_acc: 0.9000
Epoch 1836/2000
448/448 [==============================] - 0s 409us/step - loss: 0.3644 - acc: 0.8214 - val_loss: 0.3372 - val_acc: 0.9000
Epoch 1837/2000
448/448 [==============================] - 0s 405us/step - loss: 0.3754 - acc: 0.8214 - val_loss: 0.3338 - val_acc: 0.8800
Epoch 1838/2000
448/448 [==================

448/448 [==============================] - 0s 375us/step - loss: 0.3730 - acc: 0.8281 - val_loss: 0.3257 - val_acc: 0.8800
Epoch 1891/2000
448/448 [==============================] - 0s 375us/step - loss: 0.3791 - acc: 0.7969 - val_loss: 0.3289 - val_acc: 0.8800
Epoch 1892/2000
448/448 [==============================] - 0s 374us/step - loss: 0.3792 - acc: 0.8348 - val_loss: 0.3301 - val_acc: 0.8800
Epoch 1893/2000
448/448 [==============================] - 0s 374us/step - loss: 0.4063 - acc: 0.8147 - val_loss: 0.3405 - val_acc: 0.8800
Epoch 1894/2000
448/448 [==============================] - 0s 374us/step - loss: 0.3558 - acc: 0.8348 - val_loss: 0.3491 - val_acc: 0.8800
Epoch 1895/2000
448/448 [==============================] - 0s 375us/step - loss: 0.3827 - acc: 0.8214 - val_loss: 0.3452 - val_acc: 0.8800
Epoch 1896/2000
448/448 [==============================] - 0s 374us/step - loss: 0.4008 - acc: 0.8058 - val_loss: 0.3456 - val_acc: 0.8800
Epoch 1897/2000
448/448 [==================

448/448 [==============================] - 0s 377us/step - loss: 0.3638 - acc: 0.8170 - val_loss: 0.3144 - val_acc: 0.8800
Epoch 1950/2000
448/448 [==============================] - 0s 375us/step - loss: 0.3644 - acc: 0.8415 - val_loss: 0.3142 - val_acc: 0.8800
Epoch 1951/2000
448/448 [==============================] - 0s 378us/step - loss: 0.3576 - acc: 0.8259 - val_loss: 0.3285 - val_acc: 0.8600
Epoch 1952/2000
448/448 [==============================] - 0s 412us/step - loss: 0.3823 - acc: 0.8125 - val_loss: 0.3286 - val_acc: 0.8800
Epoch 1953/2000
448/448 [==============================] - 0s 394us/step - loss: 0.3590 - acc: 0.8304 - val_loss: 0.3110 - val_acc: 0.8600
Epoch 1954/2000
448/448 [==============================] - 0s 395us/step - loss: 0.3607 - acc: 0.8214 - val_loss: 0.3081 - val_acc: 0.8600
Epoch 1955/2000
448/448 [==============================] - 0s 397us/step - loss: 0.3602 - acc: 0.8147 - val_loss: 0.3055 - val_acc: 0.8600
Epoch 1956/2000
448/448 [==================

In [108]:
import tensorflow as tf
tf.VERSION

'1.13.1'